In [1]:
import numpy as np
import yaml
from onsager import OnsagerCalc
from onsager import crystal
from onsager import crystalStars as stars
from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
def build_point_group_ops(lattice, basis, threshold=1e-8):

        pgroup_ops = []
        sgroup_ops = []
        inv_lat_vec = np.linalg.inv(lattice)
        supercellvect = [np.array((n0, n1, n2))
                         for n0 in range(-1, 2)
                         for n1 in range(-1, 2)
                         for n2 in range(-1, 2)
                         if (n0, n1, n2) != (0, 0, 0)]
        nmat_list = [X for X in [np.array((n0, n1, n2))
                                 for n0 in supercellvect
                                 for n1 in supercellvect
                                 for n2 in supercellvect] if abs(np.linalg.det(X)) == 1]
        for nmat in nmat_list:
            g = np.dot(lattice, np.dot(nmat, inv_lat_vec))
            if np.all(abs(np.dot(g.T, g) - np.eye(3)) < threshold):
                flag_op = 1
                for bas in basis:
                    vec1 = bas - basis[0]
                    for i, j in enumerate(vec1):
                        if j < 0:
                            vec1[i] += 1.0
                    vec2 = np.dot(nmat, vec1)
                    for i, j in enumerate(vec2):
                        if j < 0:
                            vec2[i] += 1.0
                    if np.any(abs(vec1 - vec2) > threshold):
                        flag_op = 0
                        sgroup_ops.append(nmat)

                if flag_op:
                    pgroup_ops.append(nmat)
        return np.array(pgroup_ops), np.array(sgroup_ops) 
        

def apply_pg_site_dir(pg, site1, site2, threshold=1e-8):
        
    if np.any([np.all(abs(site1 - np.dot(g, site2)) < threshold) for g in pg]):
            return 1
    else:
            return 0

        
def apply_sg_site_dir(sg, site1, site2, threshold=1e-8):
        
    if np.any([np.all(abs(site1 - np.dot(g, site2)) < threshold) for g in sg]):
            return 1
    else:
            return 0

def apply_pg_trans(pg, trans1, trans2, threshold=1e-8):

    if np.any([np.all(abs(trans1[0] - np.dot(g, trans2[0])) < threshold) and np.all(abs(trans1[1] - np.dot(g, trans2[1])) < threshold) for g in pg]):
            return 1
    else:
            return 0

        
def apply_sg_trans(sg, trans1, trans2, threshold=1e-8):
        
    if np.any([np.all(abs(trans1[0] - np.dot(g, trans2[0])) < threshold)and np.all(abs(trans1[1] - np.dot(g, trans2[1])) < threshold) for g in sg]):
            return 1
    else:
            return 0
def read_data(data_file):
    site_data = []
    trans_data = []
    dict_loader = yaml.load(data_file)
    vacancy_data = dict_loader.get('vacancy_data')  # vacancy data  
    site_data = dict_loader.get('site_data')  # site data
    trans_data = dict_loader.get('trans_data')  # transition data
    for i, data in enumerate(trans_data):
        if len(data) == 6:
            trans_data[i].append([0, 0.0, 0.0])
    bulk = dict_loader.get('bulk')      
    for i in range(len(site_data)):
        site_data[i][2]-=bulk[1]
        site_data[i][1]=bulk[0]/site_data[i][1]
    for i in range(len(trans_data)):
        trans_data[i][3]-=bulk[1]
        trans_data[i][5]-=bulk[1]
        trans_data[i][2]=bulk[0]/trans_data[i][2]
        trans_data[i][4]=bulk[0]/trans_data[i][4]
        if trans_data[i][6][0]:
            #trans_data[i][6][1]=18.299152044/trans_data[i][6][1]
            trans_data[i][6][1]=bulk[0]/trans_data[i][6][1]
            trans_data[i][6][2]-=bulk[1]                        
    w0_data = []
    w1_data = []
    w2_data = []
    for data in trans_data:
        if np.all(data[0]==[0,0,0]):
            w0_data.append(data)
        elif np.all(data[1]==[0,0,0]):
            w2_data.append(data)
        else:
            w1_data.append(data)
    return vacancy_data, site_data, w0_data, w1_data, w2_data        

In [2]:
## get data
data = open('Be_data.yaml', 'r')
vacancy_data, site_data, w0_data, w1_data, w2_data   = read_data(data)

In [3]:
## get list of deleted states and meta states
#for state in starset.states:
#    if state.i not in meta_sites and state.j in meta_sites:
#        if np.allclose(0.866025403784,np.linalg.norm(state.dx)):
#            to_del.append(state)
deleted_states = []
meta_states = []
new_w1_data = []
for jump in w1_data:
    if (jump[-1][0]):
        meta = np.array((np.array(jump[0])+np.array(jump[1]))/2).tolist()
        meta_states.append([meta,jump[-1][1],jump[-1][2]])
        new_jump1 = [jump[0],meta,jump[2],jump[3],jump[2],jump[3]]
        new_jump2 = [jump[1],meta,jump[4],jump[5],jump[4],jump[5]]
        new_w1_data.append(new_jump1)
        new_w1_data.append(new_jump2)
    else:
        deleted_states.append((np.array(jump[0])+np.array(jump[1]))/2)
        new_w1_data.append(jump)

In [4]:
new_w1_data

[[[0.333333333, 0.666666667, -0.5],
  [0.0, 1.0, -1.0],
  5.849319734874114,
  0.6169099999999617,
  5.849319734874114,
  0.6169099999999617,
  [0, 0.0, 0.0]],
 [[0.333333333, 0.666666667, -0.5],
  [0.0, 1.0, 0.0],
  5.849319734874114,
  0.4126299999999219,
  5.849319734874114,
  0.4126299999999219,
  [0, 0.0, 0.0]],
 [[0.333333333, 0.666666667, -0.5],
  [0.0, 0.0, -1.0],
  5.849319734874114,
  0.7782499999999573,
  5.849319734874114,
  0.7782499999999573,
  [0, 0.0, 0.0]],
 [[0.333333333, 0.666666667, -0.5],
  [0.333333333, 1.166666667, -0.5],
  5.205044742388395,
  0.57716999999991,
  5.205044742388395,
  0.57716999999991],
 [[0.333333333, 1.666666667, -0.5],
  [0.333333333, 1.166666667, -0.5],
  5.205044742388395,
  0.6159399999999096,
  5.205044742388395,
  0.6159399999999096],
 [[0.333333333, 0.666666667, -0.5],
  [-0.166666667, 0.666666667, -0.5],
  5.205044742388395,
  0.6185499999999138,
  5.205044742388395,
  0.6185499999999138],
 [[-0.666666667, 0.666666667, -0.5],
  [-0.1666

In [5]:
deleted_states.append(np.array([0.5,0.5,0]))

In [6]:
a= 3.2342373809
c_a= 1.5989108537
c=a*c_a
HCP = crystal.Crystal.HCP(a0=a,c_a=c_a, chemistry="Zr")
pg,sg = build_point_group_ops(HCP.lattice/a, HCP.basis[0])
len(pg)
print(HCP)
meta_basis = HCP.Wyckoffpos(np.array([5/6,2/3,0.25]))
basis = HCP.basis[0] + meta_basis
HCPmeta = crystal.Crystal(HCP.lattice, basis[0:8], chemistry=["Zr"], noreduce=True)
sitelist = HCPmeta.sitelist(0)
vacancyjumps = HCPmeta.jumpnetwork(0, 1.01*a)
meta_sites = np.arange(2,8,1)
for pos,jlist in enumerate(vacancyjumps):
        if np.any([np.allclose(dx,a*np.array([0.5, -0.8660254, 0.])) for (i,j), dx in jlist]):
            ind1 = pos
            break
#print("ind1 = ",ind1)
for pos,jlist in enumerate(vacancyjumps):
        if np.any([np.allclose(dx,a*np.array([ 0.25, -0.4330127, 0.])) for (i,j), dx in jlist]):
            ind2 = pos
            break
#print("ind2 = ",ind2)
jumpnetwork = [vacancyjumps[1], vacancyjumps[ind2]]
jumpnetwork2 = [vacancyjumps[1], vacancyjumps[ind1]]

#Lattice:
  a1 = [ 1.61711869 -2.80093173  0.        ]
  a2 = [ 1.61711869  2.80093173  0.        ]
  a3 = [ 0.          0.          5.17125725]
#Basis:
  (Zr) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Zr) 0.1 = [ 0.66666667  0.33333333  0.75      ]


In [7]:
print(HCPmeta)

#Lattice:
  a1 = [ 1.61711869 -2.80093173  0.        ]
  a2 = [ 1.61711869  2.80093173  0.        ]
  a3 = [ 0.          0.          5.17125725]
#Basis:
  (Zr) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Zr) 0.1 = [ 0.66666667  0.33333333  0.75      ]
  (Zr) 0.2 = [ 0.83333333  0.66666667  0.25      ]
  (Zr) 0.3 = [ 0.16666667  0.83333333  0.75      ]
  (Zr) 0.4 = [ 0.83333333  0.16666667  0.25      ]
  (Zr) 0.5 = [ 0.16666667  0.33333333  0.75      ]
  (Zr) 0.6 = [ 0.33333333  0.16666667  0.25      ]
  (Zr) 0.7 = [ 0.66666667  0.83333333  0.75      ]


In [8]:
starset = stars.StarSetMeta(jumpnetwork, HCPmeta, 0, meta_sites = meta_sites)
starset.generate(4)
to_del = []
for i, state in enumerate(starset.states):
    if state.i not in meta_sites and state.j in meta_sites:
        if state.i==0: 
            if np.any([apply_pg_site_dir(pg,np.dot(HCP.invlatt,state.dx), site) for site in deleted_states]):
                to_del.append(state)
                #print(i)
        elif state.i==1: 
            if np.any([apply_sg_site_dir(sg,np.dot(HCP.invlatt,state.dx), site) for site in deleted_states]):
                to_del.append(state)                
                #print(i)

In [9]:
HCPdiffuser = OnsagerCalc.VacancyMediatedMeta(HCPmeta, 0, sitelist, jumpnetwork, 4, meta_sites = np.arange(2,8,1), jumpnetwork2= jumpnetwork2, deleted_states=to_del)

In [10]:
binding_entropy_list = []
binding_energy_list = []
for i,state in enumerate(HCPdiffuser.interactlist()):
        data_not_found = 1
        #print(i,state)
        for site in site_data:
            if state.i==0: 
                if apply_pg_site_dir(pg,np.dot(HCP.invlatt,state.dx), site[0]):
                    print(i,state,np.linalg.norm(state.dx))
                    print(site[0],site[2])
                    binding_entropy_list.append(site[1])
                    binding_energy_list.append(site[2])
                    data_not_found = 0
                    break
            else:        
                if apply_sg_site_dir(sg,np.dot(HCP.invlatt,state.dx), site[0]):
                    print(i,state,np.linalg.norm(state.dx))
                    print(site[0],site[2])
                    binding_entropy_list.append(site[1])
                    binding_energy_list.append(site[2])
                    data_not_found = 0
                    break
        if data_not_found:
            for site in meta_states:
                if state.i==0: 
                    if apply_pg_site_dir(pg,np.dot(HCP.invlatt,state.dx), site[0]):
                        print(i,state,np.linalg.norm(state.dx))
                        print(site[0],site[2])
                        binding_entropy_list.append(site[1])
                        binding_energy_list.append(site[2]-0.51727)
                        data_not_found = 0
                        break
                else:        
                    if apply_sg_site_dir(sg,np.dot(HCP.invlatt,state.dx), site[0]):
                        print(i,state,np.linalg.norm(state.dx))
                        print(site[0],site[2])
                        binding_entropy_list.append(site[1])
                        binding_energy_list.append(site[2]-0.51727)
                        data_not_found = 0
                        break
        if data_not_found:           
                    #print("no data; setting binding energy = 0")
                    if state.j in meta_sites:
                        #binding_entropy_list.append(54.169024409/18.299152044)
                        binding_entropy_list.append(1.0)
                    else:
                        binding_entropy_list.append(1.0)                    
                    binding_energy_list.append(0.0)

16 0.[0,0,0]:6.[-1,0,0] (dx=[-2.4256780356749994,1.4004658668643235,0.0]) 2.80093173373
[0.5, 1.0, 0.0] 0.5272499999999809
17 1.[0,0,0]:5.[0,-1,0] (dx=[-2.4256780356749994,-1.400465866864324,0.0]) 2.80093173373
[0.5, 1.0, 0.0] 0.5272499999999809
18 1.[0,0,0]:7.[-1,-1,0] (dx=[-2.4256780356749994,1.400465866864324,0.0]) 2.80093173373
[1.0, 0.5, 0.0] 0.09175999999990836
19 0.[0,0,0]:4.[-1,1,0] (dx=[0.0,2.8009317337286475,0.0]) 2.80093173373
[1.0, 0.5, 0.0] 0.09175999999990836
20 1.[0,0,0]:7.[1,0,0] (dx=[2.425678035675,-1.4004658668643235,0.0]) 2.80093173373
[0.5, 1.0, 0.0] 0.5272499999999809
21 0.[0,0,0]:2.[0,1,0] (dx=[2.4256780356749994,1.4004658668643244,0.0]) 2.80093173373
[0.5, 1.0, 0.0] 0.5272499999999809
22 0.[0,0,0]:6.[1,1,0] (dx=[2.425678035675,-1.400465866864324,0.0]) 2.80093173373
[1.0, 0.5, 0.0] 0.09175999999990836
23 0.[0,0,0]:4.[0,0,0] (dx=[0.0,-2.800931733728648,0.0]) 2.80093173373
[0.5, 1.0, 0.0] 0.5272499999999809
24 1.[0,0,0]:5.[1,1,0] (dx=[2.4256780356750003,1.4004658668

In [11]:
##HCPtracer = {'preV': np.array([1.0,1.0/10000000.0]), 'eneV': np.array([0.0, 0.0]),
##             'preT0': np.array([ 1.0, 1.0]),
##             'eneT0': np.array([0, 0]),
##              }

# HCPtracer = {'preV': np.array([1.0,54.169024409/18.299152044],), 'eneV': np.array([0.0,0.51727]),
#              'preT0': np.array([ 54.169024409/9.26073917, 54.169024409/10.40702378]),
#              'eneT0': np.array([0.613339999999994,0.553549999999973]),
#               }

HCPsolute = {'preV': np.array([1.0,54.169024409/18.299152044],), 'eneV': np.array([0.0,0.51727]),
             'preT0': np.array([ 54.169024409/9.26073917, 54.169024409/10.40702378]),
             'eneT0': np.array([0.613339999999994,0.553549999999973]),
              }
HCPsolute['preSV'] = np.array(binding_entropy_list)
HCPsolute['eneSV'] = np.array(binding_energy_list)
HCPsolute['preS']= np.array([1.0,1.0])
HCPsolute['eneS']= np.array([0.,0])
# HCPtracer['preT2'] = np.array([ 54.169024409/9.26073917, 0.5* 54.169024409/10.40702378,54.169024409/10.40702378])
# HCPtracer['eneT2'] = np.array([0.613339999999994,0.553549999999973,1e12])


In [12]:
HCPsolute.update(HCPdiffuser.makeLIMBpreene(**HCPsolute))
for k,v in zip(HCPsolute.keys(), HCPsolute.values()): print(k,v)

preT1 [   3.16960511    3.16960511    1.63500996    3.01731467    3.01731467
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    8.94869974
    1.52835437   18.13749818   23.52771805    2.68497169  114.3915717
   23.52771805    2.68497169   33.47168566   45.61047172   58.52102193
   23.48978007   16.51429883   26.50968332   58.52102193   91.83613912
 

In [13]:
omega2=HCPdiffuser.omegalist(2)[0]
for j, (S1,S2) in enumerate(omega2):
    data_not_found = 1
    print(S1.i,S1.dx,S2.dx)
    if S1.i==0:
        for trans in w2_data:                        
            if apply_pg_site_dir(pg,np.dot(HCP.invlatt,S1.dx), trans[0]): 
                print(trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT2'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT2'][j]= trans[2]
                data_not_found = 0
                break
    else:
        for trans in w2_data:                        
            if apply_sg_site_dir(sg,np.dot(HCP.invlatt,S1.dx), trans[0]): 
                print(trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT2'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT2'][j]= trans[2]
                data_not_found = 0
                break
    if data_not_found:
            print("no data; limb used")
            #binding_entropy_list.append(1)
            #binding_energy_list.append(0.0)   



1 [ 1.61711869 -0.93364391  2.58562863] [-1.61711869  0.93364391 -2.58562863]
[0.333333333, 0.666666667, -0.5] [0.0, 0.0, 0.0] 0.5550578493199977 0.9921899999999368
0 [ 3.23423738  0.          0.        ] [ -3.23423738e+00   1.11022302e-16   0.00000000e+00]
[1.0, 1.0, 0.0] [0.0, 0.0, 0.0] 0.5890237460274756 0.9464699999999766


In [91]:
omega1=HCPdiffuser.omegalist(1)[0]
for j, (S1,S2) in enumerate(omega1):
    data_not_found = 1    
    if S1.i==0:
        for i,trans in enumerate(new_w1_data):                        
            if apply_pg_trans(pg,np.array([np.dot(HCP.invlatt,S1.dx),np.dot(HCP.invlatt,S2.dx)]),np.array([trans[0],trans[1]])):
                #print(S1.i,S1.dx,S2.dx)
                #print(j,i,trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT1'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT1'][j]= trans[2]
                data_not_found = 0
                break
            elif apply_pg_trans(pg,np.array([np.dot(HCP.invlatt,S1.dx),np.dot(HCP.invlatt,S2.dx)]),np.array([trans[1],trans[0]])):
                #print(S1.i,S1.dx,S2.dx)
                #print(j,i,trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT1'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT1'][j]= trans[2]
                data_not_found = 0
                break
    elif S1.i==1:
        for i,trans in enumerate(new_w1_data):                              
            if apply_sg_trans(sg,np.array([np.dot(HCP.invlatt,S1.dx),np.dot(HCP.invlatt,S2.dx)]),np.array([trans[0],trans[1]])):
                #print(S1.i,S1.dx,S2.dx)
                #print(j,i,trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT1'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT1'][j]= trans[2]
                data_not_found = 0
                break
            elif apply_sg_trans(sg,np.array([np.dot(HCP.invlatt,S1.dx),np.dot(HCP.invlatt,S2.dx)]),np.array([trans[1],trans[0]])):
                #print(S1.i,S1.dx,S2.dx)
                #print(j,i,trans[0],trans[1],trans[2],trans[3])
                HCPsolute['eneT1'][j]= trans[3] + HCPsolute['eneV'][0] + HCPsolute['eneS'][0] 
                HCPsolute['preT1'][j]= trans[2]
                data_not_found = 0
                break
    if data_not_found:
            print("no data; limb used")
            if np.isclose(HCPsolute['eneT1'][j],0.55355):
                HCPsolute['preT1'][j]=5.20504474229
            print(HCPsolute['eneT1'][j],HCPsolute['preT1'][j])
            #binding_entropy_list.append(1)
            #binding_energy_list.append(0.0)      

no data; limb used
0.6196 5.84931973729
no data; limb used
0.630895 5.84931973729
no data; limb used
0.82776 5.84931973729
no data; limb used
0.70358 5.84931973729
no data; limb used
0.74379 5.84931973729
no data; limb used
0.92797 5.84931973729
no data; limb used
0.74379 5.84931973729
no data; limb used
0.755085 5.84931973729
no data; limb used
0.74379 5.84931973729
no data; limb used
0.8215 5.84931973729
no data; limb used
0.79752 5.84931973729
no data; limb used
0.624635 5.84931973729
no data; limb used
0.624635 5.84931973729
no data; limb used
0.624635 5.84931973729
no data; limb used
0.61334 5.84931973729
no data; limb used
0.61334 5.84931973729
no data; limb used
0.61334 5.84931973729
no data; limb used
0.61334 5.84931973729
no data; limb used
0.61334 5.84931973729
no data; limb used
0.61334 5.84931973729
no data; limb used
0.61334 5.84931973729
no data; limb used
0.61334 5.84931973729
no data; limb used
0.61334 5.84931973729
no data; limb used
0.61334 5.84931973729
no data; limb

In [14]:
for k,v in zip(HCPsolute.keys(), HCPsolute.values()): print(k,v)

preT1 [   3.16960511    3.16960511    1.63500996    3.01731467    3.01731467
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    5.84931974
    5.84931974    5.84931974    5.84931974    5.84931974    8.94869974
    1.52835437   18.13749818   23.52771805    2.68497169  114.3915717
   23.52771805    2.68497169   33.47168566   45.61047172   58.52102193
   23.48978007   16.51429883   26.50968332   58.52102193   91.83613912
 

In [15]:
#Temp=np.arange(500, 1010, 100)
#Temp = [500,1000]
Temp = [500]
D_Onsager=[]
for T in Temp:
    pre=1e-8 # THz and Angstrom unit scaling
    Lvv, Lss, Lsv, L1vv = HCPdiffuser.Lij(*HCPdiffuser.preene2betafree(kB*T, **HCPsolute))
    #Lss=Lss*pre
    #D_Onsager.append([Lss[0,0],Lss[1,1],Lss[2,2]])    
    #print(T, Lsv[0,0]/Lss[0,0],Lsv[1,1]/Lss[1,1],Lsv[2,2]/Lss[2,2])
    print(T, Lsv[0,0]/Lss[0,0],Lsv[1,1]/Lss[1,1],Lsv[2,2]/Lss[2,2])
    Lss=Lss*pre
    Lvv=Lvv*pre
    print(T, Lss[0,0],Lss[1,1],Lss[2,2])    
    print(T, Lvv[0,0],Lvv[1,1],Lvv[2,2])
    
D_Onsager=np.array(D_Onsager)

500 -8.94674811904 -8.94674811904 -3.33917578103
500 2.95501887524e-17 2.95501887524e-17 1.1109899233e-17
500 1.2762391424e-12 1.2762391424e-12 7.71151340391e-13


In [16]:
Temp=[float(T) for T in range(500,1023,100)]
#Temp=[float(T) for T in [500,1000]]
#Temp.append(923.)
Temp=np.array(Temp)
Dbb_Onsager=[]
Drag_Onsager=[]
Dvv_Onsager=[]
D1vv_Onsager=[]
for T in Temp:
    pre=1e-8 # THz and Angstrom unit scaling
    Lvv, Lss, Lsv, L1vv = HCPdiffuser.Lij(*HCPdiffuser.preene2betafree(kB*T, **HCPsolute))
    Lss=Lss*pre
    Lsv=Lsv*pre
    Lvv=Lvv*pre
    L1vv=L1vv*pre
    Dbb_Onsager.append([T,Lss[0,0],Lss[1,1],Lss[2,2]])
    Drag_Onsager.append([T,Lsv[0,0]/Lss[0,0],Lsv[1,1]/Lss[1,1],Lsv[2,2]/Lss[2,2]])
    Dvv_Onsager.append([Lvv[0,0],Lvv[1,1],Lvv[2,2]])
    D1vv_Onsager.append([L1vv[0,0],L1vv[1,1],L1vv[2,2]])
    
Dbb=np.array(Dbb_Onsager)
Drag=np.array(Drag_Onsager)
Dvv_Onsager=np.array(Dvv_Onsager)
D1vv_Onsager=np.array(D1vv_Onsager)

In [17]:
outfilec = open('Diffusivity-Be.dat', 'w')
outfilec.write('T(K)\t\t')
outfilec.write('Dxx\t\t')
outfilec.write('Dyy\t\t')
outfilec.write('Dzz\t\t')
outfilec.write('\n')
for i in range(len(Dbb)):
    outfilec.write('%2.5f\t' %Dbb[i,0])
    outfilec.write('%0.14g\t' %Dbb[i,1])
    outfilec.write('%0.14g\t' %Dbb[i,2])
    outfilec.write('%0.14g\t' %Dbb[i,3])
    outfilec.write('\n')
outfilec.close()
outfilec = open('Drag-Be.dat', 'w')
for i in range(len(Drag)):
    outfilec.write('%2.5f\t' %Drag[i,0])
    outfilec.write('%0.14g\t' %Drag[i,1])
    outfilec.write('%0.14g\t' %Drag[i,2])
    outfilec.write('%0.14g\t' %Drag[i,3])
    outfilec.write('\n')
outfilec.close()